In [ ]:
#Listening Client Requests
#Building Server Side Socket
#Syntax: socket(family,protocol)

from socket import *
import fileinput
import requests as req

class ResponseManager:
    def __init__(self):
        self.filename = ""
    def getBlockedUserPage(self):
        f = open("blockedip.html","r")
        str = f.read()
        return str
    def getBlockedKeywordPage(self):
        f = open("blockedkeyword.html","r")
        str = f.read()
        return str
    def getBlockedWebsitePage(self):
        f = open("blockedwebsite.html","r")
        str = f.read()
        return str

class RequestParser:
    def __init__(self,requestString):
        lines = str(requestString).splitlines()
        self.reqMethod = lines[0].split(" ")[0]
        self.reqUrl = lines[0].split(" ")[1]
        self.reqHost = lines[1].split(" ")[1]
    def getReqMethod(self):
        return self.reqMethod
    def getReqUrl(self):
        return self.reqUrl
    def getReqHost(self):
        return self.reqHost
    
    
class PythonProxyServer:
    def __init__(self,port):
        self.ip = '0.0.0.0'
        self.port = port
        self.http_header = "HTTP/1.1 200 OK\n\n"
        self.socket = socket(AF_INET, SOCK_STREAM)
        self.socket.setsockopt(SOL_SOCKET, SO_REUSEADDR,1)
        #Binding the socket
        self.socket.bind((self.ip,self.port))
        self.responseManager = ResponseManager()
        self.isValid = True
        
    def listen(self,backlog):
        self.socket.listen(backlog)
        print("Proxy Server started on port: "+str(self.port))
    
    def accept(self):
        self.responseWriter,self.clientAddr = self.socket.accept()
        self.requestStr = self.responseWriter.recv(1024).decode()
        print(self.requestStr)
        reqParser = RequestParser(self.requestStr)
        #msg = self.responseManager.getBlockedUserPage()
        #self.responseWriter.sendall(msg.encode("utf-8"))
        self.url = reqParser.getReqUrl()
        print(self.url)
        self.host = reqParser.getReqHost()
        self.isBlockedHost(self.host)
        self.isBlockedKeyword(self.url)
        self.forwardRequest()
    
    def isBlockedHost(self,hostName):
        hostNames = ['yahoo.com','xyz.com','abc.com','mno.in']
        if(hostName in hostNames):
            msg = self.responseManager.getBlockedWebsitePage()
            self.isValid = False
            self.responseWriter.sendall(msg.encode("utf-8"))
            self.responseWriter.close()
        #else:
            #msg = "Success"
            #self.responseWriter.sendall(msg.encode("utf-8"))
    
    def isBlockedKeyword(self,url):
        keywords = ['movies','xyz','abc','mno']
        strUrl = str(url)
        for word in keywords:
            if word in strUrl:
                msg = self.responseManager.getBlockedKeywordPage()
                self.isValid = False
                self.responseWriter.sendall(msg.encode("utf-8"))
                self.responseWriter.close()
            #else:
                #msg = "Success"
                #self.responseWriter.sendall(msg.encode("utf-8"))
                
    def forwardRequest(self):
        try:
            if(self.isValid==True):
                header = {
                    'User-Agent':'Mozilla/5.0 (Macintosh: Intel Mac OS X 10.12;rv:55,0)Gecko/20100101 Firefox/55.0',
                }
                resp = req.get(self.url,headers = header)
                #print(resp.content)
                self.responseWriter.sendall(resp.content)
        except Exception as e:
            print(e)

In [ ]:

proxy = PythonProxyServer(2647)
proxy.listen(5)#No.of waiting request in waiting queue is 5
while True:
    proxy.accept()